### The lottery is no-risk gamble based on Bitcoin

Generate some keys and check the balance.

In [ ]:
import requests
from bitcoinlib.keys import Key
from bitcoinlib.services.services import Service
from multiprocessing.pool import ThreadPool as Pool

In [ ]:
pool = Pool(100)

In [ ]:
def gen_keys():
    key = Key()
    return (key.private_hex, key.public_hex, key.address())

In [ ]:
def get_balance(addr, providers=[]):
    return Service(providers=providers).getbalance([addr]) / (10**8)

def get_balance(addr):
    res = requests.get(f'https://blockstream.info/api/address/{addr}').json()
    btc = (res['chain_stats']['funded_txo_sum'] - res['chain_stats']['spent_txo_sum']) / 10**8
    csh = (res['mempool_stats']['funded_txo_sum'] - res['mempool_stats']['spent_txo_sum']) / 10**8
    return (btc, csh)

In [ ]:
def get_lottery_entry():
    (priv, pub, addr) = gen_keys()
    bal_btc, bal_csh = get_balance(addr)
    return (priv, pub, addr, bal_btc, bal_csh)

In [ ]:
def lottery(n):
    def exec_lot(_):
        res = get_lottery_entry()
        print('.', end='')
        return res
    res = pool.map(exec_lot, range(0, n))
    print('')
    return res

In [ ]:
def lottery_pretty(n):
    print(f'You are drawing {n} lottery tickets (chance to win: ~1/(10^240ish))')
    res = lottery(n)
    (privs, pubs, addrs, bal_btc, bal_csh) = zip(*res)
    tot_bal = sum(bal_btc) + sum(bal_csh)
    if tot_bal:
        win_details = [x for x in res if x[3]]
        print(f'Wow! You actually won! You get {tot_bal} btc!')
        for det in win_details:
            print(f'Use the following data:')
            print(f'Private key:', det[0])
            print(f'Public key:', det[1])
            print(f'Address:', det[2])
            print(f'Balance: {det[4]} btc / {det[5]} csh')
    else:
        print(f'Sadly no win. Better luck next time!')

In [ ]:
lottery_pretty(10)

In [ ]:
test = Key('b5e4f7ad4607200c8d07eb1295cf4d9d00f06f7b32edc475dfbaa72abc7a9705')

In [ ]:
(test.private_hex, test.public_hex, test.address())